In [1]:
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
%matplotlib inline

/Users/adammarblestone/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
batch_size = 16
num_batches = 20

In [3]:
# Define an RNN representing the PFC

num_units_PFC = 10

PFC_cell = tf.contrib.rnn.LSTMBlockCell(num_units = num_units_PFC)

# This does one cycle of the RNN
def encode_step(input_data, network_state):
    with tf.variable_scope("encoder",reuse=DO_SHARE):
        return encoder_cell(inputs = input_data, state = network_state) 
    
PFC_state_previous = PFC_cell.zero_state(batch_size, tf.float32) # Initial state of PFC

In [4]:
# Define a feedforward NN representing the sensory-motor system

input_vis = tf.placeholder(shape=[batch_size, 2], dtype=tf.float32, name = 'input_vis')
input_aud = tf.placeholder(shape=[batch_size, 2], dtype=tf.float32, name = 'input_aud')
input_total = tf.concat([input_vis, input_aud], axis = -1)

num_hidden_sensorymotor = 10
hidden_sensorymotor = tf.contrib.layers.fully_connected(input_total, num_hidden_sensorymotor, activation_fn = tf.nn.relu)
num_out_sensorymotor = 2
out_sensorymotor = tf.contrib.layers.fully_connected(hidden_sensorymotor, num_out_sensorymotor, activation_fn = tf.nn.relu)

In [5]:
# Just train the sensory-motor system to ignore audition
run_sensorymotor_test = False

if run_sensorymotor_test:
    loss = tf.reduce_mean(tf.norm(out_sensorymotor-input_vis))
    learning_rate = 0.01
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op=optimizer.minimize(loss)
    
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())
    
    num_inputs = batch_size*num_batches
    
    in_vis_list_unshaped = [[np.random.rand(), np.random.rand()] for k in range(num_inputs)] 
    in_vis_list = np.reshape(in_vis_list_unshaped, [num_batches,batch_size,2])
    
    in_aud_list_unshaped = [[np.random.rand(), np.random.rand()] for k in range(num_inputs)]
    in_aud_list = np.reshape(in_aud_list_unshaped, [num_batches,batch_size,2])

    losses = []
    for i in range(num_batches):
        in_vis_list_batch = in_vis_list[i]
        in_aud_list_batch = in_aud_list[i]
        l, it, os, _ =  sess.run([loss, input_total, out_sensorymotor, train_op], feed_dict = {input_vis:in_vis_list_batch, input_aud:in_aud_list_batch})
        losses.append(l)
        
    plt.figure()
    plt.plot(losses)
    plt.title("Loss function versus number of batches")
    plt.show()

In [ ]:
# Define a feedforward NN representing the sensory-motor system with an ancillary rule label input

input_vis = tf.placeholder(shape=[batch_size, 2], dtype=tf.float32, name = 'input_vis')
input_aud = tf.placeholder(shape=[batch_size, 2], dtype=tf.float32, name = 'input_aud')
input_cue = tf.placeholder(shape=[batch_size, 1], dtype=tf.float32, name = 'input_aud')
input_total = tf.concat([input_vis, input_aud, input_cue], axis = -1)

num_hidden_sensorymotor = 10
hidden_sensorymotor = tf.contrib.layers.fully_connected(input_total, num_hidden_sensorymotor, activation_fn = tf.nn.relu)
num_out_sensorymotor = 2
out_sensorymotor = tf.contrib.layers.fully_connected(hidden_sensorymotor, num_out_sensorymotor, activation_fn = tf.nn.relu)

In [ ]:
# Train the sensory-motor system to follow the given rule
run_sensorymotor_rule_test = True

if run_sensorymotor_rule_test:    
    
    num_inputs = batch_size*num_batches
    
    in_vis_list_unshaped = [[np.random.rand(), np.random.rand()] for k in range(num_inputs)] 
    in_vis_list = np.reshape(in_vis_list_unshaped, [num_batches,batch_size,2])
    
    in_aud_list_unshaped = [[np.random.rand(), np.random.rand()] for k in range(num_inputs)]
    in_aud_list = np.reshape(in_aud_list_unshaped, [num_batches,batch_size,2])
    
    cue_list_unshaped = [[np.random.rand()] for k in range(num_inputs)]
    in_cue_list = np.reshape(cue_list_unshaped, [num_batches,batch_size,1])
                
    learning_rate = 0.01
    optimizer = tf.train.AdamOptimizer(learning_rate)
    
    sess = tf.Session()
    
    losses = []
    for i in range(num_batches):
        in_vis_list_batch = in_vis_list[i]
        in_aud_list_batch = in_aud_list[i]
        in_cue_list_batch = in_cue_list[i]
        
        loss = 0
        for j in range(batch_size):
            if in_cue_list_batch[j] > 0.5:
                loss += tf.norm(tf.gather(out_sensorymotor-input_vis, [j]))
            else:
                loss += tf.norm(tf.gather(out_sensorymotor-input_aud, [j]))
        
        train_op=optimizer.minimize(loss)
        
        sess.run(tf.initialize_all_variables())
        l, it, os, _ =  sess.run([loss, input_total, out_sensorymotor, train_op], feed_dict = {input_vis:in_vis_list_batch, input_aud:in_aud_list_batch, input_cue:in_cue_list_batch})
        losses.append(l)
        print l
        
    plt.figure()
    plt.plot(losses)
    plt.title("Loss function versus number of batches")
    plt.show()

10.3307
